# Import de librerías y preparación de data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
import glob
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
ruta = "/content/drive/My Drive/Maestria/Mineria de datos/Dataset proyecto/"

In [4]:
archivos = glob.glob(os.path.join(ruta, "*.xlsx"))
lista_df = [pd.read_excel(f) for f in archivos]
datos = pd.concat(lista_df, ignore_index=True)

In [5]:
datos.head()

,AÑO,MES,DIASESTAN,SEXO,PPERTENENCIA,EDAD,PERIODOEDA,DEPTORESIDEN,MUNIRESIDEN,CAUFIN,CONDIEGRES,TRATARECIB,DIASESTANCIA
0,2024,1,1.0,2,1,5,2,1,101,A010,1,1,NaN
1,2024,1,1.0,2,1,1,3,1,101,A010,1,1,NaN
2,2024,1,2.0,1,4,3,3,1,101,A010,1,1,NaN
3,2024,1,2.0,2,1,6,3,1,101,A010,1,1,NaN
4,2024,2,3.0,1,9,6,3,1,101,A010,1,1,NaN


In [6]:
datos_nn2 = datos.copy()
datos_nn2 = datos_nn2[
    (datos_nn2["EDAD"].between(0, 99)) &
    (datos_nn2["DIASESTAN"].between(1, 98))
]

# Crear EDAD_RANGO
def rango_edad(e):
    if e < 15:
        return "NIÑO"
    elif e < 40:
        return "ADULTO_JOVEN"
    elif e < 65:
        return "ADULTO"
    else:
        return "ADULTO_MAYOR"

datos_nn2["EDAD_RANGO"] = datos_nn2["EDAD"].apply(rango_edad)

In [9]:
# Eliminar NA en variables clave
datos_nn2 = datos_nn2.dropna(subset=["EDAD_RANGO", "SEXO", "PPERTENENCIA", "TRATARECIB", "DIASESTAN"])

X2 = datos_nn2[["DIASESTAN", "SEXO", "PPERTENENCIA", "TRATARECIB"]].copy()
y2 = datos_nn2["EDAD_RANGO"].copy()

# Codificar salida a entero
clases_rango = sorted(y2.unique())  # ['ADULTO', 'ADULTO_JOVEN', ...]
mapa_rango = {clase: idx for idx, clase in enumerate(clases_rango)}
y2_int = y2.map(mapa_rango).values

# One-hot de la salida
y2_cat = to_categorical(y2_int)

# Normalizar X2
scaler2 = StandardScaler()
X2_scaled = scaler2.fit_transform(X2)

In [10]:
# Train / Test RN2
X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2_scaled, y2_cat, test_size=0.2, random_state=42
)

# Modelo RN2 (multiclase)
model2 = Sequential()
model2.add(Dense(32, activation='relu', input_dim=X2_train.shape[1]))
model2.add(Dense(16, activation='relu'))
model2.add(Dense(y2_cat.shape[1], activation='softmax'))

model2.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
print("\n=== Entrenando RN2 – Rango de Edad ===")
history2 = model2.fit(
    X2_train, y2_train,
    epochs=25,
    batch_size=128,
    validation_split=0.2,
    verbose=1
)



=== Entrenando RN2 – Rango de Edad ===
Epoch 1/25
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.4229 - loss: 1.2394 - val_accuracy: 0.4575 - val_loss: 1.1771
Epoch 2/25
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.4597 - loss: 1.1768 - val_accuracy: 0.4613 - val_loss: 1.1743
Epoch 3/25
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.4589 - loss: 1.1730 - val_accuracy: 0.4615 - val_loss: 1.1725
Epoch 4/25
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.4607 - loss: 1.1725 - val_accuracy: 0.4632 - val_loss: 1.1721
Epoch 5/25
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.4598 - loss: 1.1740 - val_accuracy: 0.4636 - val_loss: 1.1721
Epoch 6/25
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.4596 - loss: 1.1747 - val_accuracy: 0.4622 - val_loss: 1.1725
Epoch 7/25
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.4594 - loss: 1.1725 - val_accuracy: 0.4635 - val_loss: 1.1725
Epoch 8/25
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/s

In [15]:
loss2, acc2 = model2.evaluate(X2_test, y2_test, verbose=0)
print(f"Precisión RN2 en test: {acc2:.4f}")
print(f'Loss: {loss2}, Accuracy: {acc2}')
print("Clases (índice → etiqueta):", mapa_rango)

Precisión RN2 en test: 0.4617
Loss: 1.1695839166641235, Accuracy: 0.4616931974887848
Clases (índice → etiqueta): {'ADULTO': 0, 'ADULTO_JOVEN': 1, 'ADULTO_MAYOR': 2, 'NIÑO': 3}


# Predicciones de escenarios

In [13]:
esc2_A = pd.DataFrame({
    "DIASESTAN": [1],
    "SEXO": [2],
    "PPERTENENCIA": [1],
    "TRATARECIB": [1]
})

esc2_B = pd.DataFrame({
    "DIASESTAN": [3],
    "SEXO": [2],
    "PPERTENENCIA": [2],
    "TRATARECIB": [2]
})

esc2_C = pd.DataFrame({
    "DIASESTAN": [5],
    "SEXO": [1],
    "PPERTENENCIA": [1],
    "TRATARECIB": [3]
})

In [14]:
def predecir_rn2(escenario_df, nombre):
    esc_scaled = scaler2.transform(escenario_df)
    prob = model2.predict(esc_scaled)[0]
    idx = np.argmax(prob)
    # Invertir el mapa
    inv_mapa = {v: k for k, v in mapa_rango.items()}
    clase = inv_mapa[idx]
    print(f"\nEscenario {nombre} – RN2")
    print("Probabilidades por clase:")
    for i, p in enumerate(prob):
        print(f"  {inv_mapa[i]}: {p:.3f}")
    print(f"Clasificación final: {clase}")

predecir_rn2(esc2_A, "A")
predecir_rn2(esc2_B, "B")
predecir_rn2(esc2_C, "C")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step

Escenario A – RN2
Probabilidades por clase:
  ADULTO: 0.315
  ADULTO_JOVEN: 0.220
  ADULTO_MAYOR: 0.174
  NIÑO: 0.291
Clasificación final: ADULTO
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step

Escenario B – RN2
Probabilidades por clase:
  ADULTO: 0.366
  ADULTO_JOVEN: 0.460
  ADULTO_MAYOR: 0.109
  NIÑO: 0.066
Clasificación final: ADULTO_JOVEN
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step

Escenario C – RN2
Probabilidades por clase:
  ADULTO: 0.052
  ADULTO_JOVEN: 0.942
  ADULTO_MAYOR: 0.002
  NIÑO: 0.005
Clasificación final: ADULTO_JOVEN
